# Assignment 1

You only need to write one line of code for each question. When answering questions that ask you to identify or interpret something, the length of your response doesn’t matter. For example, if the answer is just ‘yes,’ ‘no,’ or a number, you can just give that answer without adding anything else.

We will go through comparable code and concepts in the live learning session. If you run into trouble, start by using the help `help()` function in Python, to get information about the datasets and function in question. The internet is also a great resource when coding (though note that **no outside searches are required by the assignment!**). If you do incorporate code from the internet, please cite the source within your code (providing a URL is sufficient).

Please bring questions that you cannot work out on your own to office hours, work periods or share with your peers on Slack. We will work with you through the issue.

### Classification using KNN

Let's set up our workspace and use the **Wine dataset** from `scikit-learn`. This dataset contains 178 wine samples with 13 chemical features, used to classify wines into different classes based on their origin.

The **response variable** is `class`, which indicates the type of wine. We'll use all of the chemical features to predict this response variable.

In [80]:
# Import standard libraries
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [8]:
from sklearn.datasets import load_wine

# Load the Wine dataset
wine_data = load_wine()

# Convert to DataFrame
wine_df = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)

# Bind the 'class' (wine target) to the DataFrame
wine_df['class'] = wine_data.target

# Display the DataFrame
wine_df


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


#### **Question 1:** 
#### Data inspection

Before fitting any model, it is essential to understand our data. **Use Python code** to answer the following questions about the **Wine dataset**:

_(i)_ How many observations (rows) does the dataset contain?

In [14]:
# Your answer here
# .info tells the details about the dataset like number of observations of each columns, datatypes and column names.
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       178 non-null    float64
 1   malic_acid                    178 non-null    float64
 2   ash                           178 non-null    float64
 3   alcalinity_of_ash             178 non-null    float64
 4   magnesium                     178 non-null    float64
 5   total_phenols                 178 non-null    float64
 6   flavanoids                    178 non-null    float64
 7   nonflavanoid_phenols          178 non-null    float64
 8   proanthocyanins               178 non-null    float64
 9   color_intensity               178 non-null    float64
 10  hue                           178 non-null    float64
 11  od280/od315_of_diluted_wines  178 non-null    float64
 12  proline                       178 non-null    float64
 13  class

_(ii)_ How many variables (columns) does the dataset contain?

In [31]:
# Your answer here

# Total number of columns
print("Column Size of Wine Data Frame: " + str(wine_df.columns.size))

# This will display all the column names
print("\nColumn Names: ")
wine_df.columns

Column Size of Wine Data Frame: 14

Column Names: 


Index(['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium',
       'total_phenols', 'flavanoids', 'nonflavanoid_phenols',
       'proanthocyanins', 'color_intensity', 'hue',
       'od280/od315_of_diluted_wines', 'proline', 'class'],
      dtype='object')

_(iii)_ What is the 'variable type' of the response variable `class` (e.g., 'integer', 'category', etc.)? What are the 'levels' (unique values) of the variable?

In [32]:
# Your answer here

# Variable Type of 'class'
print("Variable Type of 'class': " + str(wine_df["class"].dtype))


# Unique levels of 'class'
wine_df["class"].unique

Variable Type of 'class': int64


<bound method Series.unique of 0      0
1      0
2      0
3      0
4      0
      ..
173    2
174    2
175    2
176    2
177    2
Name: class, Length: 178, dtype: int64>


_(iv)_ How many predictor variables do we have (Hint: all variables other than `class`)? 

In [42]:
# Your answer here
# Filter out the predictor variables other than 'class'
predictor_columns = wine_df.columns.drop('class')

# Size of the predictor variables
predictor_columns.size

13

You can use `print()` and `describe()` to help answer these questions.

#### **Question 2:** 
#### Standardization and data-splitting

Next, we must preform 'pre-processing' or 'data munging', to prepare our data for classification/prediction. For KNN, there are three essential steps. A first essential step is to 'standardize' the predictor variables. We can achieve this using the scaler method, provided as follows:

In [69]:
# Select predictors (excluding the last column)
predictors = wine_df.iloc[:, :-1]

# Standardize the predictors
scaler = StandardScaler()
predictors_standardized = pd.DataFrame(scaler.fit_transform(predictors), columns=predictors.columns)

# Display the head of the standardized predictors
print(predictors_standardized.head())

    alcohol  malic_acid       ash  alcalinity_of_ash  magnesium   
0  1.518613   -0.562250  0.232053          -1.169593   1.913905  \
1  0.246290   -0.499413 -0.827996          -2.490847   0.018145   
2  0.196879    0.021231  1.109334          -0.268738   0.088358   
3  1.691550   -0.346811  0.487926          -0.809251   0.930918   
4  0.295700    0.227694  1.840403           0.451946   1.281985   

   total_phenols  flavanoids  nonflavanoid_phenols  proanthocyanins   
0       0.808997    1.034819             -0.659563         1.224884  \
1       0.568648    0.733629             -0.820719        -0.544721   
2       0.808997    1.215533             -0.498407         2.135968   
3       2.491446    1.466525             -0.981875         1.032155   
4       0.808997    0.663351              0.226796         0.401404   

   color_intensity       hue  od280/od315_of_diluted_wines   proline  
0         0.251717  0.362177                      1.847920  1.013009  
1        -0.293321  0.406051

(i) Why is it important to standardize the predictor variables?

In a KNN model, the scale of each variable can significantly affect the predictions. Standardization is a preprocessing step that involves scaling and centering the data before it is fed into the model. This ensures that no single variable dominates the model due to its scale.

(ii) Why did we elect not to standard our response variable `Class`?

In a KNN model, we calculate distances between various variables using their X and Y coordinates. The response variable 'Class' is not involved in the computation or calculations so there's no need to standardize the variable 'Class'.

(iii) A second essential step is to set a random seed. Do so below (Hint: use the random.seed function). Why is setting a seed important? Is the particular seed value important? Why or why not?

Randomness is used in data analysis to make fair and unbiased decisions. The random set of data without seed might result in different output every time. Setting a seed is important because it ensures the reproducibility. Same seed will produce the same output every time of the random data.

(iv) A third essential step is to split our standardized data into separate training and testing sets. We will split into 75% training and 25% testing. The provided code randomly partitions our data, and creates linked training sets for the predictors and response variables. 

Extend the code to create a non-overlapping test set for the predictors and response variables.

In [88]:
# set a seed for reproducibility
np.random.seed(123)

# split the data into a training and testing set. hint: use train_test_split !

# Calculating the columns to scale.
standardized_wine = wine_df.copy()
columns_to_exclude = ['class']
columns_to_scale = standardized_wine.columns.difference(columns_to_exclude)

scaler = StandardScaler()
standardized_wine[columns_to_scale] = scaler.fit_transform(standardized_wine[columns_to_scale])

# Spliting the data into testing and training data.
wine_train, wine_test = train_test_split(
    standardized_wine, train_size = 0.75, test_size = 0.25
)

print("Size of Wine Train data: " + str(wine_train.size) + " -- Size of Wine Test data: " + str(wine_test.size))
print("\nStructure of Wine Train data: " + str(wine_train.shape) + " -- Structure of Wine Test data: " + str(wine_test.shape))
print("\nWINE Train Data:")
wine_train

Size of Wine Train data: 1862 -- Size of Wine Test data: 630

Structure of Wine Train data: (133, 14) -- Structure of Wine Test data: (45, 14)

WINE Train Data:


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
28,1.073917,-0.391694,1.584529,-0.028510,0.509638,1.049347,0.944462,0.065639,0.296279,-0.241413,1.283518,1.113449,0.535335,0
13,2.160950,-0.544297,0.085839,-2.430790,-0.613775,1.289697,1.667318,0.549108,2.135968,0.147900,1.283518,0.167113,1.283691,0
88,-1.680724,-0.248068,0.341713,0.632117,-1.105268,-0.552984,-0.340615,0.951998,-0.422075,-0.976782,0.186684,0.195361,-0.213021,1
24,0.616869,-0.472483,0.890014,0.151661,-0.262708,0.376368,0.583034,-0.659563,0.121071,-0.665332,0.713164,1.706675,0.312420,0
100,-1.137207,-0.230114,-2.436346,-0.599052,-0.192495,-0.104332,0.141289,-0.820719,-0.334471,-0.760497,1.371265,0.491974,-0.117486,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,1.024507,-0.687923,0.926567,0.151661,1.071345,1.049347,1.376168,0.307374,0.226196,0.666984,0.757038,-0.058878,1.220001,0
98,-0.778980,-1.136754,-0.974210,-0.298767,-0.824415,1.962676,1.727556,-0.981875,0.629175,-0.241413,0.362177,0.223610,-0.276711,1
66,0.135116,-1.190614,-2.436346,-1.349764,-1.526548,1.097417,1.155295,-0.820719,1.207363,0.104643,0.713164,0.802712,-0.779861,1
126,-0.704864,-0.723829,-0.279695,0.602088,-0.964842,0.712858,1.125176,0.226796,0.313800,-0.483652,-1.173391,0.322481,-1.257535,1


#### **Question 3:**
#### Model initialization and cross-validation
We are finally set to fit the KNN model. 


Perform a grid search to tune the `n_neighbors` hyperparameter using 10-fold cross-validation. Follow these steps:

1. Initialize the KNN classifier using `KNeighborsClassifier()`.
2. Define a parameter grid for `n_neighbors` ranging from 1 to 50.
3. Implement a grid search using `GridSearchCV` with 10-fold cross-validation to find the optimal number of neighbors.
4. After fitting the model on the training data, identify and return the best value for `n_neighbors` based on the grid search results.

In [86]:
# Initializing KNN classifier
knn = KNeighborsClassifier()
knn

# Defining a parameter grid for 'n_neighbors'
parameter_grid = {
    "n_neighbors" : range(1,51)
}

# Implementing a 'GridSearchCV' with 10-fold cross-validation
wine_tune_grid = GridSearchCV(estimator = knn, param_grid = parameter_grid, cv = 10)
wine_tune_grid.fit(predictors_standardized, wine_df['class'])

# Getting best value for n_neighbors
best_param = wine_tune_grid.best_params_
print("Getting best value for n_neighbors:" + str(best_param))

accuracy_grid = pd.DataFrame(wine_tune_grid.cv_results_)
accuracy_grid

Getting best value for n_neighbors:{'n_neighbors': 23}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002656,0.000803,0.003545,0.001340,1,{'n_neighbors': 1},0.833333,0.944444,0.944444,0.944444,1.0,0.944444,1.0,1.000000,0.882353,0.941176,0.943464,0.050459,49
1,0.001892,0.000107,0.002399,0.000071,2,{'n_neighbors': 2},0.833333,0.944444,0.944444,0.944444,1.0,0.888889,1.0,1.000000,0.882353,0.941176,0.937908,0.053038,50
2,0.001805,0.000065,0.002357,0.000072,3,{'n_neighbors': 3},0.944444,0.944444,0.944444,0.944444,1.0,0.944444,1.0,1.000000,0.882353,0.941176,0.954575,0.034784,47
3,0.001772,0.000026,0.002528,0.000520,4,{'n_neighbors': 4},0.888889,0.944444,0.944444,0.944444,1.0,0.944444,1.0,1.000000,0.882353,0.941176,0.949020,0.040003,48
4,0.001823,0.000079,0.002396,0.000089,5,{'n_neighbors': 5},0.944444,0.944444,0.944444,0.944444,1.0,0.944444,1.0,1.000000,0.941176,0.941176,0.960458,0.025916,40
5,0.001788,0.000027,0.002374,0.000047,6,{'n_neighbors': 6},0.944444,0.944444,0.888889,0.944444,1.0,0.944444,1.0,1.000000,0.941176,1.000000,0.960784,0.035739,38
6,0.001818,0.000063,0.002390,0.000113,7,{'n_neighbors': 7},1.000000,0.944444,0.944444,0.944444,1.0,0.888889,1.0,1.000000,0.941176,1.000000,0.966340,0.037061,21
7,0.001799,0.000041,0.002386,0.000062,8,{'n_neighbors': 8},0.944444,0.944444,0.944444,0.944444,1.0,0.944444,1.0,1.000000,0.941176,1.000000,0.966340,0.027500,21
8,0.001788,0.000026,0.002392,0.000046,9,{'n_neighbors': 9},0.944444,1.000000,0.944444,0.944444,1.0,0.944444,1.0,1.000000,0.941176,1.000000,0.971895,0.028120,13
9,0.001810,0.000063,0.002382,0.000097,10,{'n_neighbors': 10},0.944444,0.944444,0.944444,0.944444,1.0,0.944444,1.0,1.000000,0.941176,1.000000,0.966340,0.027500,21


#### **Question 4:**
#### Model evaluation

Using the best value for `n_neighbors`, fit a KNN model on the training data and evaluate its performance on the test set using `accuracy_score`.

In [92]:
from sklearn.metrics import accuracy_score

# Extract the best number of neighbors
best_neighbors = best_param['n_neighbors']

# Define our variables ; X and Y
X_train = wine_train.drop(columns='class')
y_train = wine_train['class']

X_test = wine_test.drop(columns='class')
y_test = wine_test['class']

# Initializing KNN classifier for best n_neighbors
knn_best = KNeighborsClassifier(n_neighbors = best_neighbors)

# Fit model with the best parameter
knn_best.fit(X_train, y_train)

# Predict on test set
wine_test_predictions = knn_best.predict(X_test)

# Calculate using accuracy score
accuracy = accuracy_score(y_test, wine_test_predictions)

print(f"Test Set Accuracy with best neighbors of {best_neighbors}: {accuracy}")


Test Set Accuracy with best neighbors of 23: 0.9333333333333333


# Criteria


| **Criteria**                                           | **Complete**                                      | **Incomplete**                                    |
|--------------------------------------------------------|---------------------------------------------------|--------------------------------------------------|
| **Data Inspection**                                    | Data is inspected for number of variables, observations and data types. | Data inspection is missing or incomplete.         |
| **Data Scaling**                                       | Data scaling or normalization is applied where necessary (e.g., using `StandardScaler`). | Data scaling or normalization is missing or incorrectly applied. |
| **Model Initialization**                               | The KNN model is correctly initialized and a random seed is set for reproducibility.            | The KNN model is not initialized, is incorrect, or lacks a random seed for reproducibility. |
| **Parameter Grid for `n_neighbors`**                   | The parameter grid for `n_neighbors` is correctly defined. | The parameter grid is missing or incorrectly defined. |
| **Cross-Validation Setup**                             | Cross-validation is set up correctly with 10 folds. | Cross-validation is missing or incorrectly set up. |
| **Best Hyperparameter (`n_neighbors`) Selection**       | The best value for `n_neighbors` is identified using the grid search results. | The best `n_neighbors` is not selected or incorrect. |
| **Model Evaluation on Test Data**                      | The model is evaluated on the test data using accuracy. | The model evaluation is missing or uses the wrong metric. |


## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Note:

If you like, you may collaborate with others in the cohort. If you choose to do so, please indicate with whom you have worked with in your pull request by tagging their GitHub username. Separate submissions are required.

### Submission Parameters:
* Submission Due Date: `11:59 PM - 05/18/2025`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/LCR/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-6-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
